<a href="https://colab.research.google.com/github/gtbnhyujmj/-Reference-BingX_Trades-Endpoints/blob/main/All_Orders_Query_the_user's_historical_orders_%5BFormal%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 所有訂單查詢（All Orders）
# 查詢用戶的歷史訂單（訂單狀態包含：已完全成交、待成交、新建、部分成交、已取消）。

In [ ]:
# API 使用主要步驟

# 查詢時間範圍最長不得超過7天
# 預設查詢近7天內的資料

In [ ]:
# GET /openApi/swap/v1/trade/fullOrder

In [ ]:
import time                   # 導入時間模組，用於產生 timestamp
import requests               # 導入 requests 模組，發送 HTTP 請求
import hmac                   # 導入 hmac 模組，計算 HMAC 簽名
from hashlib import sha256    # 導入 sha256 雜湊，用於 HMAC 簽名

In [ ]:
APIURL = "https://open-api.bingx.com"   # BingX API 入口網址
APIKEY = ""        # 輸入你的 API KEY
SECRETKEY = ""     # 輸入你的 SECRET KEY

In [ ]:
def demo():
    payload = {}   # GET 請求內容，這裡沒有額外資料
    path = '/openApi/swap/v1/trade/fullOrder'   # API 路徑：查詢歷史訂單
    method = "GET"   # HTTP 請求方法
    paramsMap = {   # API 查詢參數
        "endTime": "1702731995000",      # 查詢結束時間（毫秒時間戳）
        "limit": "500",                  # 回傳最大筆數（最多 500 筆）
        "startTime": "1702688795000",    # 查詢起始時間（毫秒時間戳）
        "symbol": "PYTH-USDT",           # 交易對
        "timestamp": "1702731995838"     # 請求時間戳（建議自動產生）
    }
    paramsStr = parseParam(paramsMap)   # 將參數 dict 轉成查詢字串
    return send_request(method, path, paramsStr, payload)   # 發送 API 請求

In [ ]:
def get_sign(api_secret, payload):
    # 用 SECRET KEY 和查詢字串產生 HMAC-SHA256 簽名
    signature = hmac.new(api_secret.encode("utf-8"), payload.encode("utf-8"), digestmod=sha256).hexdigest()
    print("sign=" + signature)  # 輸出簽名（debug 用）
    return signature

In [ ]:
def send_request(method, path, urlpa, payload):
    # 拼接完整請求 URL，加上簽名
    url = "%s%s?%s&signature=%s" % (APIURL, path, urlpa, get_sign(SECRETKEY, urlpa))
    print(url)  # 印出實際請求 URL（debug 用）
    headers = {
        'X-BX-APIKEY': APIKEY,    # APIKEY 放在 Header
    }
    # 發送 HTTP 請求
    response = requests.request(method, url, headers=headers, data=payload)
    return response.text    # 回傳 API 回應內容（純文字）

In [ ]:
def parseParam(paramsMap):
    # 將參數 dict 按 key 排序，轉換成 URL 查詢字串
    sortedKeys = sorted(paramsMap)
    paramsStr = "&".join(["%s=%s" % (x, paramsMap[x]) for x in sortedKeys])
    # 動態補上 timestamp
    if paramsStr != "":
        return paramsStr+"&timestamp="+str(int(time.time() * 1000))
    else:
        return paramsStr+"timestamp="+str(int(time.time() * 1000))

In [ ]:
if __name__ == '__main__':
    print("demo:", demo())   # 主程式測試，呼叫 demo()